In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
from scipy.stats import norm

C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorf

In [2]:
import matplotlib.pyplot as plt
from equation import PricingOption, AllenCahn, HJB
from config import PricingOptionConfig, AllenCahnConfig, HJBConfig

import time
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization

In [3]:
TF_DTYPE = tf.float32

class FeedForwardModel(object):

    def __init__(self, sess, bsde, config):
        self._sess = sess
        self.config = config
        self.bsde = bsde
        
        self.dim = bsde._dim
        self.total_time = bsde._total_time
        self.num_time_interval = bsde._num_time_interval
        self.delta_t = bsde._delta_t
                
        self.f_network = []
        self.z_network=[]
   
    def train(self):
        self.start_time = time.time()
    
        dw_valid,x_valid = self.bsde.sample(100000)
        
        feed_dict_valid = {self._x: x_valid, self._dw:dw_valid, 
                           self.lambda1: 1, self.lambda2: 0, 
                           self._is_training: False}
        
        self._sess.run(tf.global_variables_initializer())
        for step in range(self.config.num_iterations + 1):
            if step % self.config.logging_frequency == 0:
                loss, init, lower, upper = self._sess.run([self._loss, self._y_init, self._lower,self._upper], 
                                                   feed_dict=feed_dict_valid)
#                 loss, i, y, q = self._sess.run([self._loss, self._i, self._y, self._q], feed_dict=feed_dict_valid)
                elapsed_time = time.time() - self.start_time + self._t_build
                print("step: %5u,loss: %.4e,y_init: %.4e, y_lower: %.4e, y_upper: %.4e  elapsed time %3u" % (step, loss, init, lower,upper, elapsed_time))
            
            dw_train,x_train = self.bsde.sample(self.config.batch_size)
            loss=self._sess.run([self._loss ,self._train_ops], 
                                feed_dict={self._x: x_train,self._dw:dw_train,self.lambda1:1,
                                           self.lambda2:0,self._is_training: True})[0]

    def nn_structure(self, _network, _input):
        z = _network[0](_input)
        for i in range(1, len(_network)):
            z = _network[i](z)
        return z
    
    def relu_adding(self, network, unit_lst, activation = None):
        #num >=2
        num = len(unit_lst)
        network.append(Dense(units = unit_lst[0], input_shape = (self.dim,), activation = 'relu'))
        for i in range(1, num):
            network.append(Dense(units = unit_lst[i], input_shape = (unit_lst[i-1],), activation = 'relu'))
        network.append(Dense(units = self.dim, input_shape = (unit_lst[-1],), activation = activation))
    
    def build(self):
        start_time = time.time()
        time_stamp = np.arange(0, self.num_time_interval) * self.delta_t
        
        for i in range(self.num_time_interval-1):
            temp = []
            temp.append(BatchNormalization())
            self.relu_adding(temp, self.config.z_units)
            self.z_network.append(temp) 
        
        self._x = tf.placeholder(TF_DTYPE, [None,self.dim, self.num_time_interval+1], name='X')
        self._dw = tf.placeholder(TF_DTYPE, [None, self.dim, self.num_time_interval], name='dW')
        self.lambda1=tf.placeholder(TF_DTYPE, name='lambda1')
        self.lambda2=tf.placeholder(TF_DTYPE, name='lambda2')
        self._is_training = tf.placeholder(tf.bool)
        
        self._y_init = tf.Variable(tf.random_uniform([1],
                                                     minval=self.config.y_init_range[0],
                                                     maxval=self.config.y_init_range[1],
                                                     dtype=TF_DTYPE))
        self._z_init = tf.Variable(tf.random_uniform([1],
                                               minval=-.1, maxval=.1,
                                               dtype=TF_DTYPE))
        with tf.variable_scope('forward'):
            
            all_one_vec = tf.ones(shape=tf.stack([tf.shape(self._dw)[0], 1]), dtype=TF_DTYPE)
            y = all_one_vec * self._y_init
            z = all_one_vec * self._z_init
            
            i = all_one_vec * 0
            q = all_one_vec * 0
            for t in range(0, self.num_time_interval - 1):
                a_star = self.bsde.a_star(time_stamp[t], self._x[:, :, t], y, z)
                q = q + tf.exp(i) * self.bsde.f_star(time_stamp[t], self._x[:, :, t], y, z)
                i = i + self.delta_t * (a_star)
                
                y = y - self.delta_t * (
                    self.bsde.f_tf(time_stamp[t], self._x[:, :, t], y, z))
                
                y = y + tf.reduce_sum(z * self.bsde._sigma * self._x[:, :, t]
                * self._dw[:, :, t], 1, keepdims=True)

                z = self.nn_structure(self.z_network[t], self._x[:, :, t + 1])
                
            a_star = self.bsde.a_star(time_stamp[-1], self._x[:, :, -2], y, z)
            q = q + tf.exp(i) * self.bsde.f_star(time_stamp[-1], self._x[:, :, -2], y, z)
            i = i + self.delta_t * (a_star)
            temp = tf.exp(i) * self.bsde.g_tf(self.total_time, self._x[:, :, -1]) - q
            self._lower = (tf.reduce_mean(temp))
            
            y = y - self.bsde.delta_t * self.bsde.f_tf(
                time_stamp[-1], self._x[:, :, -2], y, z)
            y = y + tf.reduce_sum(z * self.bsde._sigma * self._x[:, :, -2]
                * self._dw[:, :, -1], 1, keepdims=True)
            
            loss1 = y - self.bsde.g_tf(self.total_time, self._x[:, :, -1])

            # the dual problem structure
            u=self.bsde.g_tf(self.total_time, self._x[:, :, -1])
            
            for t in range(self.num_time_interval - 1,0,-1):
                
                z=self.nn_structure(self.z_network[t-1], self._x[:, :, t])
                
                u=u+self.delta_t * (
                    self.bsde.f_tf(time_stamp[t], self._x[:, :, t], u, z))
                
                u=u-tf.reduce_sum(z * self.bsde._sigma * self._x[:, :, t]
                    * self._dw[:, :, t], 1, keepdims=True)
            
            z = all_one_vec * self._z_init
            u=u+self.delta_t * (
                    self.bsde.f_tf(time_stamp[0], self._x[:, :, 0], u, z))
                
            u=u-tf.reduce_sum(z * self.bsde._sigma * self._x[:, :, 0]
                    * self._dw[:, :, 0], 1, keepdims=True)
            
            self._upper=tf.reduce_mean(u)

            self._loss = self.lambda1 * tf.reduce_mean(tf.square(loss1))

        trainable_variables = tf.trainable_variables()
        grads = tf.gradients(self._loss, trainable_variables)
        
        global_step = tf.get_variable('global_step', [],
                                      initializer=tf.constant_initializer(0),
                                      trainable=False, dtype=tf.int32)
        learning_rate = tf.train.piecewise_constant(global_step,
                                                    self.config.lr_boundaries,
                                                    self.config.lr_values)
        
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)

        apply_op = optimizer.apply_gradients(zip(grads, trainable_variables), global_step=global_step,name='train_step')
        all_ops = [apply_op]
        self._train_ops = tf.group(*all_ops)
        self._t_build = time.time()-start_time
        

In [4]:
#params
# mu_bar=0.07
dim, total_time, num_time_interval = 5, 0.5, 10

#fit
Option= PricingOption(dim, total_time, num_time_interval)
tf.reset_default_graph()
with tf.Session() as sess:
    model = FeedForwardModel(sess, Option, PricingOptionConfig())
    model.build()
    model.train()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
step:     0,loss: 6.6458e+03,y_init: 5.7175e+00, y_lower: 5.2275e+00, y_upper: 5.3677e+00  elapsed time  15
step:  1000,loss: 2.5352e+01,y_init: 4.9148e+00, y_lower: 4.6063e+00, y_upper: 4.9250e+00  elapsed time  33
step:  2000,loss: 2.2779e+01,y_init: 4.8997e+00, y_lower: 4.6115e+00, y_upper: 4.9124e+00  elapsed time  49
step:  3000,loss: 2.0669e+01,y_init: 4.8940e+00, y_lower: 4.6017e+00, y_upper: 4.9103e+00  elapsed time  66
step:  4000,loss: 1.9791e+01,y_init: 4.8743e+00, y_lower: 4.6214e+00, y_upper: 4.9124e+00  elapsed time  85
step:  5000,loss: 1.7908e+01,y_init: 4.8732e+00, y_lower: 4.6305e+00, y_upper: 4.9113e+00  elapsed time 101
